## Assignment-07 First Step of using machine learning and models.

# step1 数据分析

In [9]:
fname = r'resource/sqlResult_1558435.csv'

In [10]:
import pandas as pd

In [11]:
news = pd.read_csv(fname, encoding='gb18030')

In [8]:
xinhua_news = news[news['source'] == '新华社']

In [13]:
len(xinhua_news) / len(news),len(xinhua_news),len(news)

(0.8778051801676133, 78661, 89611)

# step2 数据预处理

将pandas中的数据，依据是否是新华社的文章，请改变成新的数据dataframe: <content, y>, 其中，content是文章内容，y是0或者1. 你可能要使用到pandas的dataframe操作。https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html

In [12]:
news_dropna = news.dropna(subset=['source','content'])

In [11]:
len(news_dropna)

87052

In [13]:
def transform(line):
    class_ = 1 if line['source'] == '新华社' else 0
    return pd.Series([class_, line['content']], index=['y', 'content'])

In [14]:
data = news_dropna.apply(transform, axis=1)

In [15]:
corpus = data.content.to_list()

In [16]:
y = data.y.values

In [2]:
import re
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import matplotlib.pyplot as plt

In [23]:
corpus_cut = []
for sentence in corpus:
    sentence = ''.join(re.findall(r'\w+',str(sentence)))
    corpus_cut.append(' '.join(jieba.cut(sentence)))   

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.744 seconds.
Prefix dict has been built succesfully.


In [4]:
import pickle

In [ ]:
with open(r'corpus','wb') as file: #将分好词的文本保存起来，以免每次都算一次
    pickle.dump(corpus_cut,file)

In [5]:
with open(r'corpus','rb') as file: #取出之前存好的分好词的文本
    corpus_cut = pickle.load(file)

In [26]:
corpus_cut[0:3]

['此外 自 本周 6 月 12 日起 除 小米 手机 6 等 15 款 机型 外 其余 机型 已 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 以 确保 工程师 可以 集中 全部 精力 进行 系统优化 工作 有人 猜测 这 也 是 将 精力 主要 用到 MIUI9 的 研发 之中 MIUI8 去年 5 月 发布 距今已有 一年 有余 也 是 时候 更新换代 了 当然 关于 MIUI9 的 确切 信息 我们 还是 等待 官方消息',
 '骁龙 835 作为 唯一 通过 Windows10 桌面 平台 认证 的 ARM 处理器 高通 强调 不会 因为 只 考虑 性能 而 去 屏蔽掉 小 核心 相反 他们 正 联手 微软 找到 一种 适合 桌面 平台 的 兼顾 性能 和 功耗 的 完美 方案 报道 称 微软 已经 拿到 了 一些 新 的 源码 以便 Windows10 更好 地 理解 biglittle 架构 资料 显示 骁龙 835 作为 一款 集成 了 CPUGPU 基带 蓝牙 WiFi 的 SoC 比 传统 的 Wintel 方案 可以 节省 至少 30 的 PCB 空间 按计划 今年 Q4 华硕 惠普 联想 将 首发 骁龙 835Win10 电脑 预计 均 是 二合一 形态 的 产品 当然 高通 骁龙 只是 个 开始 未来 也许 还 能 见到 三星 Exynos 联发科 华为 麒麟 小米 澎湃 等 进入 Windows10 桌面 平台',
 '此前 的 一加 3T 搭载 的 是 3400mAh 电池 DashCharge 快充 规格 为 5V4A 至于 电池 缩水 可能 与 刘作 虎 所说 一加 手机 5 要 做 市面 最 轻薄 大屏 旗舰 的 设定 有关 按照 目前 掌握 的 资料 一加 手机 5 拥有 55 寸 1080P 三星 AMOLED 显示屏 6G8GBRAM64GB128GBROM 双 1600 万 摄像头 备货 量 惊喜 根据 京东 泄露 的 信息 一加 5 起 售价 是 xx99 元 应该 是 在 279928992999 中 的 某个']

# Step3: 使用tfidf进行文本向量化

参考 https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html 对

对文本进行向量化

In [6]:
vetorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=3000)
X = vetorizer.fit_transform(corpus_cut)

In [28]:
X.shape

(87052, 3000)

# Step4: 参考scikit-learning的方法，构建你的第一个机器学习模型

## Logistic Regression

In [7]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

In [17]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [18]:
x_train, x_valid,y_train,y_valid = train_test_split(x_train,y_train,test_size=0.2)

In [33]:
type(x_train),type(y_train)

(scipy.sparse.csr.csr_matrix, numpy.ndarray)

In [34]:
x_train.shape,x_valid.shape,x_test.shape

((44569, 3000), (11143, 3000), (17411, 3000))

In [35]:
lr = LogisticRegression()
lr.fit(x_train,y_train)

/home/qinhao/anaconda3/envs/py3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [36]:
lr.score(x_valid,y_valid)

0.9779233599569236

In [37]:
y_valid_pred = lr.predict(x_valid)
y_valid_pred_prob = lr.predict_proba(x_valid)

In [41]:
y_valid_pred.shape

(11143,)

In [39]:
y_valid_pred_prob

array([[2.14076403e-03, 9.97859236e-01],
       [3.40329642e-02, 9.65967036e-01],
       [8.55622581e-01, 1.44377419e-01],
       ...,
       [2.47188814e-01, 7.52811186e-01],
       [9.44045073e-04, 9.99055955e-01],
       [4.38938993e-01, 5.61061007e-01]])

In [42]:
precision_score(y_valid,y_valid_pred)

0.9788560849654098

In [43]:
recall_score(y_valid,y_valid_pred)

0.99712158808933

In [44]:
f1_score(y_valid,y_valid_pred)

0.9879044153800767

In [45]:
roc_auc_score(y_valid,y_valid_pred_prob[:,1])

0.9950921459837734

## KNN

使用默认值k=5

In [46]:
knn = KNeighborsClassifier()
knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [47]:
knn.score(x_valid, y_valid)  #x的类型为scipy.sparse.csr.csr_matrix时速度较快，x=x.toarray()以后，速度特别慢

0.8139639235394418

In [48]:
y_valid_pred = knn.predict(x_valid)
y_valid_pred_prob = knn.predict_proba(x_valid)

In [49]:
precision_score(y_valid,y_valid_pred)

0.9765364459266317

In [50]:
recall_score(y_valid,y_valid_pred)

0.8137965260545905

In [51]:
f1_score(y_valid,y_valid_pred)

0.8877700178658438

In [52]:
roc_auc_score(y_valid, y_valid_pred_prob[:, 1])

0.8834524772074608

使用grid-search寻找最优参数

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {'n_neighbors':[3,5,7,9],'weights':['uniform','distance']} #速度很慢，所以参数设置的比较少，p值，leaf值都没有设置

In [ ]:
knn_clf = KNeighborsClassifier()

In [ ]:
clf_search= GridSearchCV(knn_clf,param_grid)

In [ ]:
clf_search.fit(x_train,y_train)  #

速度很慢，没有在jupyter-notebook中执行，在ipython中执行出best_params_是n_neighbors':3,'weights':'distance'

In [53]:
knn3 = KNeighborsClassifier(n_neighbors=3,weights='distance')

In [54]:
knn3.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='distance')

## Naive Bayes

In [55]:
x_train_array = x_train.toarray()
x_valid_array = x_valid.toarray()
x_test_array = x_test.toarray()

In [56]:
nb = GaussianNB()
nb.fit(x_train_array,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [57]:
y_pred = nb.predict(x_valid_array) 
y_pred_prob = nb.predict_proba(x_valid_array)

In [58]:
nb.score(x_valid_array, y_valid)

0.8566813245984026

In [59]:
precision_score(y_valid,y_pred)

0.9957894736842106

In [60]:
recall_score(y_valid,y_pred)

0.8450620347394541

In [61]:
f1_score(y_valid, y_pred)

0.914255033557047

In [62]:
roc_auc_score(y_valid, y_pred_prob[:, 1])

0.9098939600933076

# Step6: 使用 test_data 对其进行新数据的判断, 你有没

In [65]:
lr.score(x_test,y_test)

0.9735224857848487

In [66]:
knn3.score(x_test,y_test)

0.8572167020848889

In [67]:
nb.score(x_test_array,y_test)

0.8579633564987651

In [68]:
lr_y_test_pred = lr.predict(x_test)
lr_y_test_pred_prob = lr.predict_log_proba(x_test)

In [69]:
knn3_y_test_pred = knn3.predict(x_test)
knn3_y_test_pred_prob = knn3.predict_proba(x_test)

In [70]:
nb_y_test_pred = nb.predict(x_test_array)
nb_y_test_pred_prod = nb.predict_proba(x_test_array)

In [71]:
precision_score(y_test,lr_y_test_pred)

0.9750186428038777

In [72]:
precision_score(y_test,knn3_y_test_pred)

0.9792585097925851

In [73]:
precision_score(y_test,nb_y_test_pred)

0.9958912296429105

In [74]:
recall_score(y_test,lr_y_test_pred)

0.9962537303955806

In [75]:
recall_score(y_test,knn3_y_test_pred)

0.8603720871166424

In [76]:
recall_score(y_test,nb_y_test_pred)

0.8464664423137978

In [77]:
f1_score(y_test,lr_y_test_pred)

0.985521811500895

In [78]:
f1_score(y_test,knn3_y_test_pred)

0.9159737713783546

In [79]:
f1_score(y_test,nb_y_test_pred)

0.9151192723528403

In [80]:
roc_auc_score(y_test, lr_y_test_pred_prob[:, 1])

0.9940845097111967

In [81]:
roc_auc_score(y_test, knn3_y_test_pred_prob[:, 1])

0.8829658086136005

In [82]:
roc_auc_score(y_test, nb_y_test_pred_prod[:, 1])

0.9113139114748293

# Step9: 找出所以预测为 1， 但是实际为 0 的文章。 作为抄袭的候选者。

In [83]:
y_pred_all = lr.predict(X)
y_pred_all.shape,y.shape

((87052,), (87052,))

In [84]:
data.head()

,y,content
0,0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...
1,0,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...
2,0,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...
3,1,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n
4,0,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...


In [85]:
data.shape

(87052, 2)

In [86]:
data['y_pred']=y_pred_all

In [87]:
plagiarized_news = data[(data.y == 0) & (data.y_pred == 1)]

In [88]:
len(plagiarized_news)

1628

In [89]:
plagiarized_news

,y,content,y_pred
16,0,九成以上的源包用逐位对应的方式构建，未来版本的 Debian 还将提供验证包的工具和元数据；...,1
25,0,当地时间2017年6月17日，土耳其马尼萨省，在土耳其第1步兵训练大队司令部军营，590名土...,1
32,0,其他规格方面，搭载飞思卡尔IMX6SL处理器，内存512MB，存储8GB，采用定制版系统，低...,1
37,0,切尔西球星法布雷加斯正与女友达涅拉-塞曼享受假期，在个人的INSTAGRAM上，小法发布...,1
45,0,在天山北麓，辽阔的伊犁大草原上，水草丰茂，土地肥沃，自古就出骏马。天山出天马，新疆伊犁哈萨克...,1
51,0,戈壁的大漠黄沙曾掩埋了无数西域古道，而如今一条大漠天路正顽强地与黄沙“搏斗”，在乌兰布和、腾...,1
56,0,很快，不少人主动添加记者为好友，询问是否需要扫描软件，并声称这些扫描软件能够攻破摄像头的IP...,1
61,0,在QQ搜索栏输入“摄像头 破解”，就跳出了众多相关聊天群，记者随机加入了几个，发现聊天的内容...,1
63,0,2017年上海国际电影电视节，包括6月12日至16日举办的第二十三届上海电视节和6月17日至...,1
70,0,发现???共享单车“花式载人”??挡泥板当“后座”\r\n如今，共享单车种类繁多，有的共享单...,1


In [90]:
type(plagiarized_news)

pandas.core.frame.DataFrame

In [91]:
index = plagiarized_news.index.to_list()

In [104]:
plagiarized_news.iloc[10]

y                                                          0
content    中新网6月18日电 手术后，开迪热亚和弟弟地里夏提重新看清了这七彩的世界。5月19日，12岁...
y_pred                                                     1
Name: 71, dtype: object

# 使用第4课讲解的 edit distance，在涉嫌抄袭的文章中，找到其重复的文字与被修改过的文字。

In [95]:
def edit_distance(str1, str2):
    copy = []
    matrix = [[ i + j for j in range(len(str2) + 1)] for i in range(len(str1) + 1)]
    for i in range(1, len(str1)+1):
        for j in range(1, len(str2)+1):
            if(str1[i-1] == str2[j-1]):
                d = 0
            else:
                d = 1   
            matrix[i][j] = min(matrix[i-1][j]+1, matrix[i][j-1]+1, matrix[i-1][j-1]+d)
    return matrix[len(str1)][len(str2)]

In [96]:
edit_distance('abc','abdd')

2

In [105]:
a = ''.join(re.findall(r'\w+',data.content[index[10]]))

In [155]:
dis_min = 10000

In [156]:
k = 0

#以第一条判断为抄袭的新闻为例，查找和他最相似的新闻

In [116]:
from scipy.spatial.distance import cosine

In [117]:
def distance(v1, v2): return cosine(v1, v2)

# Step11: 利用第8课讲述的新模型，进行操作，感受其中不同的参数、模型对性能的影响。

## SVM

In [19]:
from sklearn.svm import SVC

In [21]:
svc = SVC()

In [22]:
svc.fit(x_train,y_train)

/home/qinhao/anaconda3/envs/py3/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [23]:
svm_y_pred = svc.predict(x_valid)
svm_y_pred_proba = svc.decision_function(x_valid)

In [24]:
svc.score(x_valid,y_valid)

0.9035824538732141

In [25]:
precision_score(y_valid, svm_y_pred)

0.9035824538732141

In [26]:
recall_score(y_valid, svm_y_pred)

1.0

In [27]:
f1_score(y_valid, svm_y_pred)

0.9493494248538562

In [28]:
roc_auc_score(y_valid, svm_y_pred_proba)

0.9873245562710238

改变核函数

In [31]:
svc2 = SVC(kernel='linear')

In [32]:
svc2.fit(x_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [35]:
svc2_y_pred = svc2.predict(x_valid)
svc2_y_pred_proba = svc2.decision_function(x_valid)

In [36]:
svc2.score(x_valid,y_valid)

0.988441381290832

In [38]:
precision_score(y_valid,svc2_y_pred)

0.9909902789852209

In [39]:
recall_score(y_valid,svc2_y_pred)

0.9962656920387732

In [40]:
f1_score(y_valid,svc2_y_pred)

0.9936209833987083

In [41]:
roc_auc_score(y_valid,svc2_y_pred_proba)

0.995503815358672

表现有所提高

# Random Forest

In [42]:
from sklearn.ensemble import RandomForestClassifier

In [43]:
rf = RandomForestClassifier()

In [44]:
rf.fit(x_train,y_train)

/home/qinhao/anaconda3/envs/py3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [45]:
rf.score(x_valid,y_valid)

0.9883695886280423

In [58]:
rf_y_pred = rf.predict(x_valid)
rf_y_pred_prob = rf.predict_proba(x_valid)

In [61]:
precision_score(y_valid,rf_y_pred)

0.9923125693453796

In [62]:
recall_score(y_valid,rf_y_pred)

0.9948355315429843

In [63]:
f1_score(y_valid,rf_y_pred)

0.9935724488176481

In [65]:
roc_auc_score(y_valid,rf_y_pred_prob[:, 1])

0.9947815174562525

# KMeans

In [84]:
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score

In [71]:
k_means = KMeans(n_clusters=2)

In [93]:
k_y = k_means.fit(x_train)

In [94]:
k_y_pred = k_y.labels_

In [95]:
k_y_pred

array([0, 1, 1, ..., 1, 1, 1], dtype=int32)

In [97]:
calinski_harabasz_score(x_train.toarray(), k_y_pred)

1605.7377522672048

### 任务描述

报社等相关的机构，往往会遇到一个问题，就是别人家的机构使用自己的文章但是并没有标明来源。 在本次任务中，我们将解决新华社的文章被抄袭引用的问题。

给定的数据集合中，存在一些新闻预料，该预料是来自新华社，但是其来源并不是新华社，请设计技巧学习模型解决该问题。

![](https://timgsa.baidu.com/timg?image&quality=80&size=b9999_10000&sec=1566105348906&di=ee9a2de91207767364853d4decc6cca3&imgtype=0&src=http%3A%2F%2Fmmbiz.qpic.cn%2Fmmbiz_png%2FTicO2kbP6Ao5sCsSQDpehZiczLdC6hDCNvoicjcOCEKX2bLxBc9gVOw28zHyFibfIWq9ceRibP6HDTKReGkr6YyTfQQ%2F640%3Fwx_fmt%3Dpng)

### Step1: 数据分析

请在课程的GitHub中下载数据集，然后使用pandas进行读取。

In [ ]:
# it‘s your time

### Step2: 数据预处理

将pandas中的数据，依据是否是新华社的文章，请改变成新的数据dataframe: <content, y>, 其中，content是文章内容，y是0或者1. 你可能要使用到pandas的dataframe操作。https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html

In [ ]:
# it's your time

# Step3: 使用tfidf进行文本向量化

参考 https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html 对

对文本进行向量化

### Step4: 参考scikit-learning的方法，构建你的第一个机器学习模型

+ 按照课程讲解的内容，把数据集分割为 traning_data, validation_data, test_data. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

+ 参照scikit learning的示例，从构建你的第一个KNN机器学习模型。

### Step5: 在traning_data, validation_data, test_data 上观察其相关metric: recall, precision, f1等， 并解释其含义. 

In [ ]:
# it's your time

### Step6: 使用 test_data 对其进行新数据的判断, 你有没

In [ ]:
# it's your time

### Step7: 调整不同的参数，观察变化

In [ ]:
# it's your time

### Step8: 不断改变参数，直到性能达到“某个”点。问：“某个”怎么定义？

In [ ]:
# it's your time

### Step9: 找出所以预测为 1， 但是实际为 0 的文章。 作为抄袭的候选者。

In [ ]:
# it's your time

### Step10： 总结该过程，什么是数据思维？什么是机器学习思维？

### (Optional)使用第4课讲解的 edit distance，在涉嫌抄袭的文章中，找到其重复的文字与被修改过的文字。 

### Step11: 利用第8课讲述的新模型，进行操作，感受其中不同的参数、模型对性能的影响。